In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scipy.optimize import least_squares

# Load the dataset
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/withouticorvalcheack.xlsx'
data = pd.read_excel(file_path)

# Separate features (X) and target variable (y)
X = data.drop(columns=['SleepDisorder'])  # Features
y = data['SleepDisorder']  # Target

# Split data into train (70%), test (15%), and validation (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp)

# Scale the features for better performance of the neural network
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Function to build the MLP model
def build_model(input_dim):
    model = Sequential([
        Dense(10, input_dim=input_dim, activation='sigmoid'),  # Hidden layer 1
        Dense(3, activation='sigmoid'),  # Hidden layer 2
        Dense(1, activation='sigmoid')  # Output layer
    ])
    return model

# Loss function for Levenberg-Marquardt optimization
def loss_function(weights, X, y, model):
    # Reshape the flattened weights back into the correct structure for the model
    offset = 0
    reshaped_weights = []
    for layer_weights in model.get_weights():
        layer_shape = layer_weights.shape
        layer_size = np.prod(layer_shape)
        reshaped_weights.append(weights[offset:offset + layer_size].reshape(layer_shape))
        offset += layer_size

    # Set the model weights
    model.set_weights(reshaped_weights)

    # Compute predictions and loss
    predictions = model.predict(X, verbose=0).flatten()
    return y - predictions  # Return residuals for least_squares

# Optimize the model using Levenberg-Marquardt
def optimize_levenberg_marquardt(X_train, y_train, model):
    # Flatten the initial weights
    initial_weights = np.concatenate([w.flatten() for w in model.get_weights()])

    # Run optimization
    result = least_squares(
        fun=lambda w: loss_function(w, X_train, y_train, model),
        x0=initial_weights,
        method='lm'
    )

    # Reshape the optimized weights and set them in the model
    offset = 0
    reshaped_weights = []
    for layer_weights in model.get_weights():
        layer_shape = layer_weights.shape
        layer_size = np.prod(layer_shape)
        reshaped_weights.append(result.x[offset:offset + layer_size].reshape(layer_shape))
        offset += layer_size

    model.set_weights(reshaped_weights)
    return model

# Wrapper-based feature selection
def wrapper_feature_selection(X_train, y_train, X_val, y_val, num_features=3):
    feature_indices = range(X_train.shape[1])
    best_features = []
    best_accuracy = 0

    for r in range(1, num_features + 1):  # Test subsets of size 1 to num_features
        for subset in combinations(feature_indices, r):
            # Subset the features
            X_train_subset = X_train[:, subset]
            X_val_subset = X_val[:, subset]

            # Build and optimize the model
            model = build_model(input_dim=len(subset))
            optimized_model = optimize_levenberg_marquardt(X_train_subset, y_train, model)

            # Evaluate the model
            predictions = optimized_model.predict(X_val_subset, verbose=0).flatten()
            accuracy = np.mean((predictions.round() == y_val))

            print(f"Features: {subset}, Accuracy: {accuracy:.4f}")

            # Update the best features and accuracy
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_features = subset

    return best_features, best_accuracy

# Perform Wrapper-based feature selection
best_features, best_accuracy = wrapper_feature_selection(X_train, y_train, X_val, y_val, num_features=5)
print(f"Best Features: {best_features}, Best Accuracy: {best_accuracy:.4f}")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
